In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import os
import multiprocessing as mp
import ctypes

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

# 整体思路

1. 给每条路编号
2. buffer
3. 找每条道路中心点
4. 对每条轨迹，获取轨迹中间点
5. 根据轨迹中间点到路中心点的距离对所有路进行排序
6. 对轨迹上的 GPS 点，挨个 match

In [53]:
df_roads=gpd.GeoDataFrame(pd.read_pickle("../data/df_main_roads.pkl"))

df_roads

,id,geometry,center
0,"(598153574, 2767949724, 0)","POLYGON ((114.17454 22.56222, 114.17454 22.562...","(114.17621206809773, 22.560903081902268)"
1,"(2767949724, 2768006811, 1)","POLYGON ((114.17919 22.55897, 114.17919 22.558...","(114.1800173748672, 22.55824867513279)"
2,"(598153574, 297121337, 0)","POLYGON ((114.17464 22.56211, 114.17464 22.562...","(114.20008177180853, 22.56214954703517)"
3,"(297121337, 2320506360, 0)","POLYGON ((114.22842 22.56748, 114.22842 22.567...","(114.2279936303635, 22.568082525145726)"
4,"(297121337, 1824553511, 0)","POLYGON ((114.22840 22.56757, 114.22966 22.567...","(114.2413088443007, 22.5733112943007)"
...,...,...,...
16148,"(9148457351, 2754531888, 0)","POLYGON ((114.01602 22.64837, 114.01602 22.648...","(114.0161446934389, 22.6480433065611)"
16149,"(9148457261, 7202299527, 0)","POLYGON ((114.00864 22.64826, 114.00861 22.648...","(114.00952040000087, 22.648972200000863)"
16150,"(9148457386, 6327148773, 0)","POLYGON ((114.01136 22.66332, 114.01132 22.663...","(114.01180144535755, 22.66300115464245)"
16151,"(9148457385, 9148457383, 0)","POLYGON ((114.01237 22.66246, 114.01326 22.661...","(114.01335557989087, 22.66127547010913)"


In [8]:
%%bash

cd ../data/taxi/processed
ls -l | head


total 1506636
-rw-rw-r-- 1 cys cys  51889 Oct 12 17:13 粤B0334Y.csv
-rw-rw-r-- 1 cys cys  98148 Oct 12 17:06 粤B0994Y.csv
-rw-rw-r-- 1 cys cys  81721 Oct 12 17:15 粤B0C0M7.csv
-rw-rw-r-- 1 cys cys  87617 Oct 12 17:07 粤B0C0P3.csv
-rw-rw-r-- 1 cys cys  78128 Oct 12 17:16 粤B0C0P5.csv
-rw-rw-r-- 1 cys cys  71403 Oct 12 17:05 粤B0C0Q5.csv
-rw-rw-r-- 1 cys cys  97699 Oct 12 17:11 粤B0C0Q9.csv
-rw-rw-r-- 1 cys cys  44873 Oct 12 17:04 粤B0C1P1.csv
-rw-rw-r-- 1 cys cys  57152 Oct 12 17:09 粤B0C1P6.csv


测试

---

In [60]:
df_taxi=pd.read_csv("../data/taxi/processed/粤B0334Y.csv")
df_taxi["gps_time"]=pd.to_datetime(df_taxi["gps_time"])

# df_taxi.loc[(df_taxi==0).any(axis=1)]

# df_taxi=df_taxi.loc[(df_taxi!=0).all(axis=1)]
df_taxi

,lat,lng,gps_time
0,22.547550,114.057976,2019-12-02 00:00:42
1,22.547485,114.053720,2019-12-02 00:01:42
2,22.547443,114.052284,2019-12-02 00:01:57
3,22.548770,114.051100,2019-12-02 00:02:22
4,22.549063,114.051180,2019-12-02 00:02:57
...,...,...,...
1294,22.626580,113.809980,2019-12-02 23:58:57
1295,22.626580,113.809980,2019-12-02 23:58:58
1296,22.626556,113.810036,2019-12-02 23:59:01
1297,22.626272,113.810165,2019-12-02 23:59:10


gps 时间间隔加大到 30s

resample

判断距离 过小就跳过

In [37]:
(df_taxi.iloc[1]["gps_time"]-df_taxi.iloc[0]["gps_time"])<pd.Timedelta(seconds=30)

False

In [54]:
# df_taxi.resample(rule="30S", on="gps_time").sum()

In [62]:
# coords=[]
# last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-30)
# for index, row in df_taxi.iterrows():
#     if row["gps_time"]-last_time<pd.Timedelta(seconds=30):
#         continue
#     coords.append([row["lat"], row["lng"]])
#     last_time=row["gps_time"]
# coords
# len(coords)

[[22.54755, 114.057976],
 [22.547485, 114.05372],
 [22.54877, 114.0511],
 [22.549063, 114.05118],
 [22.549303, 114.05101],
 [22.549314, 114.04764],
 [22.549292, 114.04509],
 [22.549356, 114.044495],
 [22.55121, 114.06252],
 [22.550152, 114.06776],
 [22.54198, 114.06859],
 [22.542738, 114.07125],
 [22.539494, 114.07412],
 [22.540085, 114.07762],
 [22.537748, 114.07755],
 [22.537043, 114.07749],
 [22.536362, 114.07731],
 [22.534876, 114.07638],
 [22.533537, 114.07527],
 [22.53195, 114.06312],
 [22.532532, 114.06063],
 [22.535496, 114.05786],
 [22.537329, 114.058105],
 [22.537327, 114.0583],
 [22.53755, 114.05383],
 [22.537477, 114.05182],
 [22.537422, 114.05022],
 [22.537407, 114.04992],
 [22.53741, 114.04976],
 [22.537424, 114.0497],
 [22.537422, 114.04946],
 [22.537422, 114.04946],
 [22.537422, 114.04946],
 [22.537382, 114.04937],
 [22.53737, 114.04938],
 [22.53736, 114.04922],
 [22.537449, 114.04899],
 [22.537462, 114.04892],
 [22.537535, 114.04867],
 [22.537512, 114.04863],
 [22.5374

769

In [63]:
index_drop_list=[]
last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-30)
for index, row in df_taxi.iterrows():
    if row["gps_time"]-last_time<pd.Timedelta(seconds=30):
        index_drop_list.append(index)
        continue
    last_time=row["gps_time"]
df_taxi=df_taxi.drop(index_drop_list)
df_taxi
coords=df_taxi[["lat", "lng"]].values
coords
len(coords)

midpoint=coords[int(len(coords)/2)]
midpoint

,lat,lng,gps_time
0,22.547550,114.057976,2019-12-02 00:00:42
1,22.547485,114.053720,2019-12-02 00:01:42
3,22.548770,114.051100,2019-12-02 00:02:22
4,22.549063,114.051180,2019-12-02 00:02:57
5,22.549303,114.051010,2019-12-02 00:03:33
...,...,...,...
1291,22.627054,113.809715,2019-12-02 23:55:28
1292,22.627054,113.809715,2019-12-02 23:56:13
1293,22.626806,113.809780,2019-12-02 23:58:12
1294,22.626580,113.809980,2019-12-02 23:58:57


array([[ 22.54755 , 114.057976],
       [ 22.547485, 114.05372 ],
       [ 22.54877 , 114.0511  ],
       ...,
       [ 22.626806, 113.80978 ],
       [ 22.62658 , 113.80998 ],
       [ 22.625362, 113.80923 ]])

769

array([ 22.569162, 114.06937 ])

In [55]:
def get_distance(row, midpoint):
    row["distance"]=abs(row["center"][0]-midpoint[1])+abs(row["center"][1]-midpoint[0])

    return row

df_roads=df_roads.apply(get_distance, args=(midpoint,), axis=1)
df_roads=df_roads.sort_values("distance")

df_roads

,id,geometry,center,distance
3619,"(504640707, 2787061089, 0)","POLYGON ((114.06913 22.56926, 114.06913 22.569...","(114.0692799443102, 22.569180194310206)",0.000108
3621,"(2787061089, 2787061351, 0)","POLYGON ((114.07029 22.56935, 114.07030 22.569...","(114.07016757982953, 22.56925067017046)",0.000886
15281,"(8280639220, 3671791987, 0)","POLYGON ((114.07158 22.56975, 114.07157 22.569...","(114.07028096125612, 22.569653711256123)",0.001403
3618,"(504640691, 504640707, 0)","POLYGON ((114.06762 22.56822, 114.06763 22.568...","(114.06826252535376, 22.568594375353772)",0.001675
3608,"(875924264, 504640681, 0)","POLYGON ((114.06859 22.56920, 114.06858 22.569...","(114.06822734322932, 22.56972525677068)",0.001706
...,...,...,...,...
462,"(1118822753, 6601929956, 0)","POLYGON ((114.55603 22.64973, 114.55602 22.649...","(114.5539184239814, 22.65018507694028)",0.565572
461,"(1118822753, 6482134220, 0)","POLYGON ((114.55925 22.65190, 114.55926 22.651...","(114.56574287152765, 22.65183995123187)",0.579051
468,"(6482134220, 1118822753, 0)","POLYGON ((114.57380 22.65049, 114.57379 22.650...","(114.56574287152765, 22.65183995123187)",0.579051
467,"(6482134220, 6482134219, 0)","POLYGON ((114.57462 22.64940, 114.57477 22.649...","(114.57430105859574, 22.64979659140426)",0.585566


In [56]:
index_list=df_roads.index.values
index_list

geom_list=df_roads["geometry"].values
geom_list

array([ 3619,  3621, 15281, ...,   468,   467,  2465])

<GeometryArray>
 ...
Length: 16153, dtype: geometry

In [57]:
track=[]
for coord in coords:
    for i in range(len(geom_list)):
        if geom_list[i].contains(Point(coord[1], coord[0])):
            if len(track)==0 or (len(track)>0 and track[-1]!=index_list[i]):
                track.append(index_list[i])
            break
        
track

[7326,
 6901,
 7320,
 3587,
 7793,
 7535,
 3551,
 3523,
 7490,
 4599,
 3594,
 3595,
 3568,
 5999,
 6000,
 7281,
 7278,
 7263,
 7262,
 7254,
 7253,
 7260,
 4133,
 5708,
 4803,
 3410,
 3407,
 3447,
 7264,
 7271,
 7255,
 7281,
 7276,
 7279,
 3484,
 3481,
 3551,
 3536,
 3537,
 7045,
 7047,
 7523,
 7521,
 3617,
 8805,
 10049,
 10059,
 10057,
 8259,
 7959,
 8309,
 10351,
 8448,
 14812,
 8448,
 16117,
 10015,
 10016,
 9476,
 12848,
 9503,
 9504,
 9511,
 9572,
 8490,
 8491,
 267,
 8402,
 8375,
 8377,
 8382,
 11829,
 11830,
 11832,
 13575,
 11743,
 13574,
 11761,
 1279,
 11728,
 8715,
 11818,
 8702,
 9647,
 9658,
 9539,
 8476,
 12972,
 12969,
 9624,
 9633,
 9621,
 13581,
 8570,
 9950,
 8509,
 8442,
 8274,
 9960,
 12359,
 12360,
 8239,
 10120,
 9565,
 10118,
 8229,
 10297,
 10296,
 8215,
 12425,
 7218,
 6096,
 7571,
 6097,
 6253,
 5029,
 7011,
 3531,
 7021,
 5654,
 7297,
 7178,
 7256,
 7282,
 7613,
 3460,
 5654,
 7261,
 3504,
 3465,
 7390,
 7348,
 7391,
 7407,
 7351,
 7345,
 3496,
 3486,
 7279,


---

多线程

In [84]:
# global vars

def to_shared_array(arr, ctype):
    shared_array = mp.Array(ctype, arr.size, lock=False)
    temp = np.frombuffer(shared_array, dtype=arr.dtype)
    temp[:] = arr.flatten(order='C')
    return shared_array

def to_numpy_array(shared_array, shape):
    '''Create a numpy array backed by a shared memory Array.'''
    arr = np.ctypeslib.as_array(shared_array)
    return arr.reshape(shape)

lock=mp.Lock()
np_mat=np.zeros((9, len(df_roads), len(df_roads)), dtype=np.int32)
shared_mat=to_shared_array(np_mat, ctypes.c_int32)
trans_matrix=to_numpy_array(shared_mat, np_mat.shape)

In [81]:
# def create2darray(n, m):
#     mp_arr=mp.Array("i", n*m)
#     arr=np.frombuffer(mp_arr.get_obj(), ctypes.c_int)  # mp_arr and arr share the same memory
#     # make it two-dimensional
#     print('np array len=', len(arr))
#     res=arr.reshape((n, m))  # res and arr share the same memory
#     return res

# trans_matrix=[[0]*len(df_roads) for _ in range(len(df_roads))]
# trans_matrix=np.zeros((len(df_roads), len(df_roads)))

def get_distance(row, midpoint):
    row["distance"]=abs(row["center"][0]-midpoint[1])+abs(row["center"][1]-midpoint[0])

    return row

def get_seconds(row):
    row["time_seconds"]=row["gps_time"].hour*3600+row["gps_time"].minute*60+row["gps_time"].second
    return row

def resample_coords(df_taxi):
    index_drop_list=[]
    last_time=df_taxi.iloc[0]["time_seconds"]-30
    for index, row in df_taxi.iterrows():
        if row["time_seconds"]-last_time<30:
            index_drop_list.append(index)
            continue
        last_time=row["time_seconds"]
    df_taxi=df_taxi.drop(index_drop_list)
    coords=df_taxi[["lat", "lng"]].values
    time_list=df_taxi["time_seconds"].values

    return df_taxi, coords, time_list

def get_track(file_path):
    global df_roads

    df_taxi=pd.read_csv(file_path)
    df_taxi["gps_time"]=pd.to_datetime(df_taxi["gps_time"])
    df_taxi=df_taxi.apply(get_seconds, axis=1)

    df_taxi, coords, time_list=resample_coords(df_taxi)
    midpoint=coords[int(len(coords)/2)]

    df_roads_temp=df_roads.apply(get_distance, args=(midpoint,), axis=1)
    df_roads_temp=df_roads_temp.sort_values("distance")

    index_list=df_roads_temp.index.values
    geom_list=df_roads_temp["geometry"].values

    track=[]
    last_road=-1
    for i in range(len(coords)):
        coord=coords[i]
        find=False
        for j in range(len(geom_list)):
            if geom_list[j].contains(Point(coord[1], coord[0])):
                road=index_list[j]
                track.append(road)
                if last_road!=-1:
                    with lock:
                        trans_matrix[0, last_road, road]+=1
                        trans_matrix[int(time_list[i]/10800)+1, last_road, road]+=1
                        # print(last_road, road, trans_matrix[last_road, road])
                find=True
                last_road=road
                break
        if not find:
            track.append(None)

    df_taxi["track"]=track
    df_taxi.to_csv("../data/taxi_after_proc/tracks/{}".format(file_path.split("/")[-1]), index=False)
    
    print("../data/taxi_after_proc/tracks/{}".format(file_path.split("/")[-1]))
        

10min 或 15min 时间长度

In [85]:
pool=mp.Pool(16)

for file in os.listdir("../data/taxi_after_proc/merged/")[:30]:
    file_path="../data/taxi_after_proc/merged/"+file
    date=int(file.split("-")[1].split("_")[0])
    if date>=2 and date<=7:
        res=pool.apply_async(get_track, (file_path,))

pool.close()
pool.join()

../data/taxi_after_proc/tracks/12-11_粤BD99105.csv


---

In [90]:
for file in os.listdir("../data/taxi_after_proc/merged/")[:1]:
    file_path="../data/taxi_after_proc/merged/"+file
    date=int(file.split("-")[1].split("_")[0])
    print(date)
    get_track(file_path)

11


In [79]:
pd.read_csv("../data/taxi_after_proc/tracks/12-11_粤BD99105.csv")[100:200]

,lat,lng,gps_time,speed,direction,car_status,time_seconds,track
100,22.547660,114.101326,2019-12-11 07:51:38,34,101,512,28298,NaN
101,22.551468,114.099495,2019-12-11 07:53:01,19,179,512,28381,7182.0
102,22.558403,114.099290,2019-12-11 07:54:37,22,178,512,28477,6089.0
103,22.558550,114.093480,2019-12-11 07:56:08,1,136,512,28568,NaN
104,22.558556,114.092760,2019-12-11 07:57:39,24,135,512,28659,NaN
...,...,...,...,...,...,...,...,...
195,22.546180,113.942970,2019-12-11 10:48:08,4,40,512,38888,NaN
196,22.546247,113.943480,2019-12-11 10:49:09,14,44,512,38949,NaN
197,22.546185,113.944570,2019-12-11 10:49:53,10,60,512,38993,4578.0
198,22.543293,113.944900,2019-12-11 10:50:24,51,93,512,39024,4582.0


In [87]:
trans_matrix[4, 4578, 4582]
np.count_nonzero(trans_matrix)
np.count_nonzero(trans_matrix[0])
trans_matrix[:1]

1

497

248

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=int32)

In [33]:
print(len(os.listdir("../data/taxi_after_proc/merged")))

os.listdir("../data/taxi_after_proc/merged")[:30]

252072


['12-11_粤BD99105.csv',
 '12-07_粤BDD5290.csv',
 '12-03_粤BDK3582.csv',
 '12-10_粤BD91409.csv',
 '12-03_粤BD04241.csv',
 '12-11_粤BDK6038.csv',
 '12-09_粤BDB7131.csv',
 '12-05_粤BD03542.csv',
 '12-08_粤BD07247.csv',
 '12-02_粤BD45539.csv',
 '12-04_粤BDK9220.csv',
 '12-09_粤BD41837.csv',
 '12-02_粤BD25538.csv',
 '12-11_粤BD54505.csv',
 '12-04_粤BDR5692.csv',
 '12-02_粤BD73507.csv',
 '12-03_粤BDD1206.csv',
 '12-07_粤B3X6G7.csv',
 '12-12_粤BD54060.csv',
 '12-02_粤BDA7178.csv',
 '12-06_粤BD04318.csv',
 '12-11_粤BD41257.csv',
 '12-01_粤BD40443.csv',
 '12-07_粤BDN0255.csv',
 '12-03_粤BDM6673.csv',
 '12-10_粤BDD1975.csv',
 '12-01_粤BDG0129.csv',
 '12-10_粤BDC3438.csv',
 '12-09_粤BDA5172.csv',
 '12-06_粤BD93409.csv']

In [34]:
df_temp=pd.read_csv("../data/taxi_after_proc/merged/12-01_粤BD40443.csv")

df_temp

,lat,lng,gps_time,speed,direction,car_status
0,22.624868,113.808525,2019-12-01 00:05:19,33,118,512
1,22.624199,113.807200,2019-12-01 00:05:34,31,120,512
2,22.623735,113.806640,2019-12-01 00:05:51,20,85,512
3,22.622670,113.807230,2019-12-01 00:06:04,30,75,512
4,22.621271,113.807970,2019-12-01 00:06:18,52,74,512
...,...,...,...,...,...,...
922,23.024214,113.750710,2019-12-01 23:17:32,14,127,512
923,23.023796,113.751700,2019-12-01 23:18:17,5,85,512
924,23.023510,113.751656,2019-12-01 23:18:38,13,101,512
925,23.024656,113.749210,2019-12-01 23:21:02,26,141,512


In [51]:
df_temp["gps_time"]=pd.to_datetime(df_temp["gps_time"])

df_temp["gps_time"][900].second

17